In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%writefile .env

# TAVILY_API_KEY = your_key
# JINA_API_KEY = your_key
# CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
# BROWSER_HEADLESS=False

Overwriting .env


# Planning 부터 시작

In [3]:
import sys
from src.workflow import run_agent_workflow

In [4]:
# Choose workflow type: 'regular' or 'scm'
workflow_type = 'scm'  # Change to 'regular' for normal workflow

# SCM queries - Supply Chain Management examples
scm_user_query = '''
Analyze the impact of Chicago port strikes on our company's supply chain. 
Focus on lead time increases, transportation cost impacts, and potential effects on order fulfillment rates.
How should we adjust our logistics strategy and what alternative routes should we consider?
'''

scm_user_query = '''
시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
리드타임 증가, 운송비 영향, 그리고 주문 이행률에 미칠 잠재적 영향에 집중해 주세요.
우리는 물류 전략을 어떻게 조정해야 하며, 어떤 대체 경로를 고려해야 할까요?
./data 디렉토리에 우히 회사 관련 데이터가 있어. 그중 data_descriptions.txt는 각 데이터에 대한 설명이야. 
scm_researcher_node, scm_insight_analyzer_node 이건 매우매우 간단히 해줘. planner 테스트 중이거든
'''

# Regular analysis queries
regular_user_query = '''
AWS Cloud Market Analysis: Growth Prospects in Korea (2025-2030) 조사해줘. 결과는 pdf로 만들어줘, 
research와 coder는 정말 간단하게 1번씩만 호출 하고 조사도 아주 간단하게만 해. reporter 에이전트 테스트 중이거든
'''

# Select query based on workflow type
user_query = scm_user_query if workflow_type == 'scm' else regular_user_query

print(f"Selected workflow: {workflow_type}")
print(f"Query: {user_query[:100]}...")

# Other example queries (commented out)
'''
Other SCM example queries:
- "Analyze the impact of Suez Canal blockage on global semiconductor supply chain"
- "Assess the effects of Chinese port lockdowns on automotive industry supply chains"
- "Evaluate alternative shipping routes due to Red Sea disruptions"
- "Supply chain risk assessment for electronics manufacturing due to Taiwan tensions"

Other regular analysis queries:
- "LG 스타일러와 삼성 제품간의 비교가 필요해. 결과는 pdf로 만들어줘"
- "Transformer 알고리즘과 현 시점에서 어떤식으로 변화 발전 하는지 알려줘"
- "5-7세 아동을 위한 미술학원(홈스쿨)을 운영예정이야. 나는 미술치료와 심리학을 전공했어. 이런 전문성을 접목한 브랜딩, 마케팅 방법, 운영방안, 구체적인 프로그램(월별)을 기획해줘. 그리고 브랜드 이름도 추천해줘."
'''

Selected workflow: scm
Query: 
시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
리드타임 증가, 운송비 영향, 그리고 주문 이행률에 미칠 잠재적 영향에 집중해 주세요.
우리는 물류 전략을 ...


'\nOther SCM example queries:\n- "Analyze the impact of Suez Canal blockage on global semiconductor supply chain"\n- "Assess the effects of Chinese port lockdowns on automotive industry supply chains"\n- "Evaluate alternative shipping routes due to Red Sea disruptions"\n- "Supply chain risk assessment for electronics manufacturing due to Taiwan tensions"\n\nOther regular analysis queries:\n- "LG 스타일러와 삼성 제품간의 비교가 필요해. 결과는 pdf로 만들어줘"\n- "Transformer 알고리즘과 현 시점에서 어떤식으로 변화 발전 하는지 알려줘"\n- "5-7세 아동을 위한 미술학원(홈스쿨)을 운영예정이야. 나는 미술치료와 심리학을 전공했어. 이런 전문성을 접목한 브랜딩, 마케팅 방법, 운영방안, 구체적인 프로그램(월별)을 기획해줘. 그리고 브랜드 이름도 추천해줘."\n'

In [5]:
import os
import shutil
import nest_asyncio

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

nest_asyncio.apply()
remove_artifact_folder()

# Execute based on workflow type
if workflow_type == 'scm':
    print("🔗 Using SCM specialized workflow")
    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )
    
    # Print SCM workflow history
    print("\n=== SCM Analysis History ===")
    print("result", result)
    for history in result.get("history", []):
        print("===")
        print(f'agent: {history["agent"]}')
        print(f'message: {history["message"]}')
        
    # Show artifacts summary for SCM
    print(f"\n📁 Check ./artifacts/ folder for generated analysis files")
    if os.path.exists("./artifacts/"):
        files = [f for f in os.listdir("./artifacts/") if f.endswith('.txt')]
        print(f"Generated {len(files)} analysis files:")
        for file in sorted(files):
            print(f"  - {file}")

else:
    print("📊 Using regular analysis workflow")
    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )

    # Print the conversation history
    print("\n=== Conversation History ===")
    print("result", result)
    for history in result["history"]:
        print("===")
        print(f'agent: {history["agent"]}')
        print(f'message: {history["message"]}')


INFO [src.workflow] ===== Starting SCM workflow =====

INFO [src.workflow] 
SCM user input: 
시카고 항만 파업이 우리 회사 공급망에 미치는 영향을 분석해 주세요.
리드타임 증가, 운송비 영향, 그리고 주문 이행률에 미칠 잠재적 영향에 집중해 주세요.
우리는 물류 전략을 어떻게 조정해야 하며, 어떤 대체 경로를 고려해야 할까요?
./data 디렉토리에 우히 회사 관련 데이터가 있어. 그중 data_descriptions.txt는 각 데이터에 대한 설명이야. 
scm_researcher_node, scm_insight_analyzer_node 이건 매우매우 간단히 해줘. planner 테스트 중이거든


INFO [src.graph.scm_nodes] ===== SCM Researcher starting task =====

INFO [src.utils.strands_sdk_utils] SCM_RESEARCHER - Prompt Cache Disabled


'./artifacts/' 폴더를 삭제합니다...
'./artifacts/' 폴더가 성공적으로 삭제되었습니다.
🔗 Using SCM specialized workflow
I'll research the Chicago port strike situation and its supply chain impacts, focusing on lead times, transportation costs, and order fulfillment rates. I'll conduct this research in both English and Korean to ensure comprehensive coverage.

First, let me check if we have any existing research context: 
## Calling tool: python_repl_tool - # Cycle: 1



INFO [python_repl_tool] ===== Executing Python code =====

INFO [python_repl_tool] ===== Code execution successful =====

INFO [src.tools.decorators] Coder - Successfully executed:

```python
import os

# Check for existing research context
results_file = './artifacts/01_research_results.txt'

if os.path.exists(results_file):
    print("Found existing research file. Reading previous context...")
    try:
        with open(results_file, 'r', encoding='utf-8') as f:
            existing_content = f.read()
        
        print("=== EXISTING RESEARCH CONTEXT ===")
        print(existing_content)
        print("=== END OF EXISTING CONTEXT ===")
        
    except Exception as e:
        print(f"Error reading existing context: {e}")
else:
    print("No existing research file found. Starting fresh research.")
```


INFO [src.tools.decorators] 
Stdout: No existing research file found. Starting fresh research.



Let me start with searching for information about the Chicago port strike and its impacts: 
## Calling tool: tavily_tool - # Cycle: 2



INFO [tavily_tool] ===== Searching for: Chicago port strike 2025 supply chain impact shipping delays =====

INFO [tavily_tool] ===== Search successful =====

INFO [src.tools.decorators] 
Coder - Tool handle_tavily_tool returned:


# Relative Search Results

[{"title": "US port strike by 45,000 dockworkers is all but ... - Chicago Sun-Times", "url": "https://chicago.suntimes.com/transportation/2024/09/30/us-port-maritime-alliance-strike-dockworkers-longshoremen-certain-begin-midnight", "content": "\"If the strikes go ahead, they will cause enormous delays across the supply chain, a ripple effect which will no doubt roll into 2025 and cause chaos across the industry,\" noted Jay Dhokia"}, {"title": "Why are the longshoremen striking? What the strike will ... - NBC Chicago", "url": "https://www.nbcchicago.com/news/why-are-longshoreman-dockworkers-striking-what-will-the-strike-affect-list-of-ports/3561862/", "content": "\"If the strikes go ahead, they will cause enormous delays across the

...

 
## Calling tool: file_read - # Cycle: 2


╔══════════════════════════════════════════════ 📄 02_data_desc.txt ══════════════════════════════════════════════╗
║                                                                                                                 ║
║     1                                                                                                           ║
║     2 ==================================================                                                        ║
║     3 # SCM Data Feasibility Assessment - 2025-06-23 12:41:08                                                   ║
║     4 --------------------------------------------------                                                        ║
║     5                                                                                                           ║
║     6 ## Problem Statement                                                                                      ║
║     7 시카고 항만 파업이 공급망에 미치는 영향과 대응 방안 분석이 필요합니다. 특히 리드타임 증가, 운송비 영향,   ║
║     8                                                                                                           ║
║     9 ## Research Impact Summary                                                                                ║
║    10 연구 결과에서 확인된 주요 영향:                                                                           ║
║    11 1. 리드타임 증가 및 배송 지연 [1][2]                                                                      ║
║    12 2. 대체 경로 사용으로 인한 운송비 상승 [4]                                                                ║
║    13 3. 전반적인 물류 혼란으로 인한 주문 이행 영향 [1][3]                                                      ║
║    14                                                                                                           ║
║    15 ## Dataset Inventory                                                                                      ║
║    16 사용 가능한 데이터셋:                                                                                     ║
║    17 1. supplier_performance_data.txt                                                                          ║
║    18    - 공급업체 성과 지표                                                                                   ║
║    19    - 납기 준수율, 리드타임, 품질 점수 등 포함                                                             ║
║    20                                                                                                           ║
║    21 2. shipment_tracking_data.txt                                                                             ║
║    22    - 선적 추적 정보                                                                                       ║
║    23    - 운송 경로, 리드타임, 운송 비용 등 포함                                                               ║
║    24                                                                                                           ║
║    25 3. order_fulfillment_data.txt                                                                             ║
║    26    - 주문 이행 데이터                                                                                     ║
║    27    - 납기 준수, 주문 가치, 페널티 정보 등 포함                                                            ║
║    28                                                                                                           ║
║    29 4. inventory_levels_data.txt                                                                              ║
║    30    - 재고 수준 데이터                                                                                     ║
║    31    - 재고량, 안전재고, 재주문 트리거 등 포함                                                              ║
║    32                                                                                                           ║
║    33 5. ira_compliance_data.txt                                                                                ║
║    34    - IRA 규정 준수 데이터                                                                                 ║
║    35    - 원산지, 가공 위치, 중국 함유율 

이제 연구 결과와 데이터 설명을 토대로 포괄적인 공급망 분석 계획을 수립하겠습니다.

# Plan
## thought
- 연구 결과에 따르면 시카고 항만 파업이 공급망 전반에 심각한 영향을 미칠 것으로 예상됨
- 주요 영향 영역: 리드타임 증가, 운송비 상승, 주문 이행률 저하
- 데이터 분석 가능 여부를 검토한 결과, 중점 분석 요청 사항(리드타임, 운송비, 주문 이행률)에 대한 분석이 가능함
- 기존 데이터셋(supplier_performance_data.txt, shipment_tracking_data.txt, order_fulfillment_data.txt, inventory_levels_data.txt)을 활용하여 KPI 영향 분석 실시 가능
- 대체 경로 분석은 제한된 데이터로 부분적으로 가능하며, 재고 영향 분석도 일부 제한된 범위 내에서 가능함
## title: 시카고 항만 파업의 공급망 영향 분석 계획
## steps:
### 1. scm_impact_analyzer: 주요 KPI 영향 정량화
- [ ] ./artifacts/01_research_results.txt와 ./artifacts/02_data_desc.txt를 검토하여 파업의 영향과 가용 데이터셋 이해
- [ ] shipment_tracking_data.txt를 활용하여 리드타임 증가 영향 정량화 (배송 시간 변화율, 평균 지연일 등)
- [ ] shipment_tracking_data.txt의 transport_cost_usd, route_type 데이터를 활용하여 운송비 증가 영향 분석
- [ ] order_fulfillment_data.txt를 활용하여 주문 이행률 영향 계산 (이행률 감소 예측, 납기 준수율 등)
- [ ] supplier_performance_data.txt를 활용하여 공급업체 성과 영향 분석 (납기 준수율 변화 등)
- [ ] 각 KPI별 영향을 시나리오별(최선/가능성 높음/최악)로 추정하여 정량적 범위 제시

### 2. scm_correlation_an


INFO [src.graph.scm_nodes] ===== Planner completed task =====

INFO [src.workflow] ===== SCM Workflow completed successfully =====

INFO [src.workflow] Generated 2 analysis files in ./artifacts/



=== SCM Analysis History ===
result {'TEAM_MEMBERS': ['scm_impact_analyzer', 'scm_correlation_analyzer', 'scm_mitigation_planner', 'reporter'], 'full_plan': 'Let me read the research findings and dataset descriptions to create a comprehensive supply chain impact analysis plan.이제 연구 결과와 데이터 설명을 토대로 포괄적인 공급망 분석 계획을 수립하겠습니다.\n\n# Plan\n## thought\n- 연구 결과에 따르면 시카고 항만 파업이 공급망 전반에 심각한 영향을 미칠 것으로 예상됨\n- 주요 영향 영역: 리드타임 증가, 운송비 상승, 주문 이행률 저하\n- 데이터 분석 가능 여부를 검토한 결과, 중점 분석 요청 사항(리드타임, 운송비, 주문 이행률)에 대한 분석이 가능함\n- 기존 데이터셋(supplier_performance_data.txt, shipment_tracking_data.txt, order_fulfillment_data.txt, inventory_levels_data.txt)을 활용하여 KPI 영향 분석 실시 가능\n- 대체 경로 분석은 제한된 데이터로 부분적으로 가능하며, 재고 영향 분석도 일부 제한된 범위 내에서 가능함\n## title: 시카고 항만 파업의 공급망 영향 분석 계획\n## steps:\n### 1. scm_impact_analyzer: 주요 KPI 영향 정량화\n- [ ] ./artifacts/01_research_results.txt와 ./artifacts/02_data_desc.txt를 검토하여 파업의 영향과 가용 데이터셋 이해\n- [ ] shipment_tracking_data.txt를 활용하여 리드타임 증가 영향 정량화 (배송 시간 변화율, 평균 지연일 등)\n- [ ] shipment_trac

# SCM (Supply Chain Management) Analysis

이 섹션은 공급망 분석 전용 워크플로입니다. 

## SCM 워크플로 특징:
- **전문 에이전트**: SCM 전문가 에이전트들이 단계별 분석 수행
- **파일 기반**: 각 단계별 분석 결과를 `./artifacts/` 폴더에 저장
- **연구 출처**: 모든 연구 결과에 URL 출처 포함
- **KPI 분석**: 공급망 KPI 정량 분석 및 상관관계 분석
- **대응 전략**: 구체적인 공급망 리스크 대응 방안 제시

## 주요 분석 영역:
- 항만 파업, 운하 봉쇄 등 물류 중단 이벤트
- 운송비 증가, 리드타임 연장 등 비용 영향
- 공급업체 다변화, 대체 루트 등 대응 전략

In [ ]:
# SCM Quick Test - 간단한 SCM 분석 테스트
import os
import shutil
import nest_asyncio
from src.workflow import run_scm_workflow

# 빠른 SCM 테스트 쿼리들
scm_queries = {
    "chicago_port": "Analyze the impact of Chicago port strikes on our supply chain",
    "suez_canal": "Assess the supply chain impact of Suez Canal disruptions on electronics industry", 
    "china_lockdown": "Evaluate the effects of Chinese port lockdowns on automotive supply chains",
    "red_sea": "Analyze alternative shipping routes due to Red Sea security concerns"
}

# 테스트할 쿼리 선택
selected_query = "chicago_port"  # 원하는 쿼리 키로 변경
user_query = scm_queries[selected_query]

print(f"🔗 Testing SCM query: {selected_query}")
print(f"📝 Query: {user_query}")

# 아티팩트 폴더 정리
def remove_artifact_folder(folder_path="./artifacts/"):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Cleared {folder_path}")

nest_asyncio.apply()
remove_artifact_folder()

# SCM 워크플로 실행
result = run_scm_workflow(
    user_input=user_query,
    debug=False
)

print("\n✅ SCM Analysis Complete!")
print(f"📁 Check ./artifacts/ folder for generated files")